# C CodeGen

将isl的ir重新生成到c, 算是理论联系实际中非常重要的一环, 但是目前大部分教程并没有这块内容. 因此我决定将其补齐.
通过学习ppcg的代码, 我们需要pet/isl一起使用, 同时还需要使用到isl中一些未导出的类和函数. 

isl导出函数比较简单: 首先需要在类/函数前加上`__isl_export`即可, 然后isl提供了一个extract_interface的程序通过解析isl源码生成出对应的c++/python接口(注意部分类型导出到cpp时会需要特殊处理), 因此直接执行`make isl.py`即可更新python接口. 
注意isl在导出接口时对参数的使用做了规范化,如果是callback函数`__isl_keep`需要进行copy, 如果普通函数`__isl_take`就需要copy. 虽然isl部分的接口导出都是自动的, 但是pet的接口导出就需要自己注意.

In [1]:
import pet
import isl

def parse_code(source: str, func_name: str):
  with open("/tmp/parse_code.c", "w") as f:
    f.write(source)
  scop = pet.scop.extract_from_C_source("/tmp/parse_code.c", func_name)
  context = scop.get_context()
  schedule = scop.get_schedule()
  reads = scop.get_may_reads()
  writes = scop.get_may_writes()
  return (scop, context, schedule, reads, writes)


scop, context, schedule, reads, writes = parse_code("""
void foo()
{
	int i;
	int a;

#pragma scop
	for (i = 0; i < 10; ++i)
		a = 5;
#pragma endscop
}
""", "foo")

class CSource():
  def __init__(self, path: str) -> None:
    with open(path, 'r') as f:
      self.context = f.read()
  def _repr_html_(self) -> str:
    return "<pre class='code'><code class=\"cpp hljs\">" + self.context + "</code></pre>"


## 默认CodeGen流程

在isl中, 首先是将schedule转换为ast(通过ast_build), 接着遍历ast打印为代码. 如果不加任何修改, 打印输出为如下:

In [2]:


printer = isl.printer.from_file('/tmp/1.c')
builder = isl.ast_build()
tree: isl.ast_node = builder.node_from(schedule)
options = isl.ast_print_options.alloc()
tree.print(printer, options)
printer.flush()

CSource('/tmp/1.c')

## Ast Print CallBack

ast_print_options支持两种callback函数, 用于在遍历ast的过程中, 自定义处理需要的信息:
```python
options.set_print_for(callback)
options.set_print_user(callback)
```

比如我们可以在print for的时候添加基于openmp的并行:

In [3]:
def print_for_callback(p: isl.printer, opt: isl.ast_print_options, node: isl.ast_node_for):
  # when loop can parallel execute:
  p.start_line()
  p.print_str("#pragma omp parallel for")
  p.end_line()
  node.print(p, opt)
  return p


printer = isl.printer.from_file('/tmp/2.c')
options = isl.ast_print_options.alloc()
options = options.set_print_for(print_for_callback)
tree.print(printer, options)
printer.flush()
CSource('/tmp/2.c')


为了让打印出来的代码符合c的形式, 可以自定义print user:

In [4]:
def print_user_callback(p: isl.printer, opt: isl.ast_print_options, node: isl.ast_node_user):
  # when loop can parallel execute:
  p = p.start_line()
  p = p.print_str(isl.ast_expr.to_C_str(node.expr()))
  p = p.print_str(";")
  p = p.end_line()
  return p

printer = isl.printer.from_file('/tmp/3.c')
options = isl.ast_print_options.alloc()
# options = options.set_print_for(print_for_callback)
options = options.set_print_user(print_user_callback)
tree.print(printer, options)
printer.flush()
CSource('/tmp/3.c')

注意到这里输出的for循环的结果还是不符合c的格式, 那是因为我们没有设定isl printer的format:

In [5]:

def print_user_callback(p: isl.printer, opt: isl.ast_print_options, node: isl.ast_node_user):
  # when loop can parallel execute:
  id = node.annotation()
  p = p.start_line()
  p = p.print_str(isl.ast_expr.to_C_str(node.expr()))
  p = p.print_str(";")
  p = p.end_line()
  return p

printer = isl.printer.from_file('/tmp/4.c')
printer.set_output_format(isl.ISL_FORMAT.C)
options = isl.ast_print_options.alloc()
# options = options.set_print_for(print_for_callback)
options = options.set_print_user(print_user_callback)
tree.print(printer, options)
printer.flush()
CSource('/tmp/4.c')

# CodeGen With Stmt.

注意到上面输出的代码中每个statement都是用named id来代替表示的, 这样并不符合c语言的形式. 所以我们需要将这些named id和源代码中的statement对应起来.


1. ppcg在`ast_build`中设定了`at_each_domain`的callback, 在每个domain中通过statement的id找到pet解析的scop中对应的stmt, 并设定annotation.
2. ppcg 在`print_user`中通过`isl_ast_node_get_annotation`获取annotation, 然后在通过annotation获得stmt, 最后直接调用pet的print stmt.

In [6]:
def find_stmt_from_scop(id: isl.id) -> pet.stmt:
  """ 在pet解析的scop中找到对应的stmt.  """
  n_stmt = scop.get_n_stmt()
  for i in range(n_stmt):
    stmt = scop.get_stmt(i)
    domain = stmt.get_domain()
    id_i = domain.get_tuple_id()
    if (id.ptr == id_i.ptr):
      return stmt
 
id_dict = dict()

def at_each_domain(node: isl.ast_node_user, build: isl.ast_build):
  expr: isl.ast_expr_op = node.get_expr()
  arg: isl.ast_expr_id = expr.get_arg(0)
  id: isl.id = arg.get_id()
  stmt: pet.stmt = find_stmt_from_scop(id)
  map = build.get_schedule().as_map()
  map = map.reverse()
  iterator_map = map.as_pw_multi_aff()

  def pullback_index(index: isl.multi_pw_aff, id: isl.id):
    return index.pullback(iterator_map)

  ref2expr = stmt.build_ast_exprs(build, pullback_index, None)
  id_dict[id.ptr] = (stmt, ref2expr)

  return node.set_annotation(id)


def print_user(p: isl.printer, opt: isl.ast_print_options, node: isl.ast_node_user):
  # when loop can parallel execute:
  id = node.annotation()
  (stmt, ref2expr) = id_dict[id.ptr]
  stmt: pet.stmt
  p = stmt.print_body(p, ref2expr)
  return p


printer = isl.printer.from_file('/tmp/5.c')
printer.set_output_format(isl.ISL_FORMAT.C)
builder = isl.ast_build()
builder = builder.set_at_each_domain(at_each_domain)
tree: isl.ast_node = builder.node_from(schedule)
options = isl.ast_print_options.alloc()
options = options.set_print_user(print_user)
tree.print(printer, options)
printer.flush()

CSource('/tmp/5.c')
